## 10. Collaboration

### 90 Consider Static Analysis via `typing` to Obviate Bugs

In [1]:
import logging

In [2]:
try:
    def subtract(a, b):
        return a - b
    
    subtract(10, '5')
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-2-88bc7dade399>", line 5, in <module>
    subtract(10, '5')
  File "<ipython-input-2-88bc7dade399>", line 3, in subtract
    return a - b
TypeError: unsupported operand type(s) for -: 'int' and 'str'


```python
# ex02.py

def subtract(a: int, b: int) -> int:
    return a - b

subtract(10, '5')  # Oops: passed string value
```

```shell
$ python -m mypy --strict item_90_example_02.py
item_90_example_02.py:25: error: Argument 2 to "subtract" has incompatible type "str"; expected "int"
Found 1 error in 1 file (checked 1 source file)
```

In [3]:
try:
    def concat(a, b):
        return a + b
    
    concat('first', b'second')
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-3-a103692a641f>", line 5, in <module>
    concat('first', b'second')
  File "<ipython-input-3-a103692a641f>", line 3, in concat
    return a + b
TypeError: can only concatenate str (not "bytes") to str


```python
# ex04.py

def concat(a: str, b: str) -> str:
    return a + b

concat('first', b'second')  # Oops: passed bytes value
```

```shell
$ python -m mypy --strict item_90_example_04.py
item_90_example_04.py:25: error: Argument 2 to "concat" has incompatible type "bytes"; expected "str"
Found 1 error in 1 file (checked 1 source file)
```

In [4]:
class Counter:
    def __init__(self):
        self.value = 0

    def add(self, offset):
        value += offset

    def get(self) -> int:
        self.value

In [5]:
try:
    counter = Counter()
    counter.add(5)
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-5-8066bc7fddec>", line 3, in <module>
    counter.add(5)
  File "<ipython-input-4-39042fef683e>", line 6, in add
    value += offset
UnboundLocalError: local variable 'value' referenced before assignment


In [6]:
try:
    counter = Counter()
    found = counter.get()
    assert found == 0, found
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-6-43078c99d68d>", line 4, in <module>
    assert found == 0, found
AssertionError: None


```python
# ex08.py

class Counter:
    def __init__(self) -> None:
        self.value: int = 0  # Field / variable annotation

    def add(self, offset: int) -> None:
        value += offset      # Oops: forgot "self."

    def get(self) -> int:
        self.value           # Oops: forgot "return"

counter = Counter()
counter.add(5)
counter.add(3)
assert counter.get() == 8
```

```shell
$ python -m mypy --strict item_90_example_08.py
item_90_example_08.py:27: error: Name 'value' is not defined
item_90_example_08.py:29: error: Missing return statement
Found 2 errors in 1 file (checked 1 source file)
```

In [7]:
def combine(func, values):
    assert len(values) > 0
    
    result = values[0]
    for next_value in values[1:]:
        result = func(result, next_value)
    
    return result

def add(x, y):
    return x + y

try:
    inputs = [1, 2, 3, 4j]
    result = combine(add, inputs)
    assert result == 10, result  # Fails
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-7-d644350be35a>", line 16, in <module>
    assert result == 10, result  # Fails
AssertionError: (6+4j)


```python
# ex10.py

from typing import Callable, List, TypeVar

Value = TypeVar('Value')
Func = Callable[[Value, Value], Value]

def combine(func: Func[Value], values: List[Value]) -> Value:
    assert len(values) > 0

    result = values[0]
    for next_value in values[1:]:
        result = func(result, next_value)

    return result

Real = TypeVar('Real', int, float)

def add(x: Real, y: Real) -> Real:
    return x + y

inputs = [1, 2, 3, 4j]  # Oops: included a complex number
result = combine(add, inputs)
assert result == 10
```

```shell
$ python -m mypy --strict item_90_example_10.py
item_90_example_10.py:42: error: Argument 1 to "combine" has incompatible type "Callable[[Real, Real], Real]"; expected "Callable[[complex, complex], complex]"
Found 1 error in 1 file (checked 1 source file)
```

In [8]:
def get_or_default(value, default): 
    if value is not None:
        return value
    return value

try:
    found = get_or_default(3, 5)
    assert found == 3
    
    found = get_or_default(None, 5)
    assert found == 5, found  # Fails
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-8-c54465e3f8ce>", line 11, in <module>
    assert found == 5, found  # Fails
AssertionError: None


```python
# ex12.py

from typing import Optional

def get_or_default(value: Optional[int],
                   default: int) -> int: 
    if value is not None:
        return value
    return value  # Oops: should have returned "default"
```

```shell
$ python -m mypy --strict item_90_example_12.py
item_90_example_12.py:28: error: Incompatible return value type (got "None", expected "int")
Found 1 error in 1 file (checked 1 source file)
```

> 파이썬 `typing` 모듈은 예외를 인터페이스 정의의 일부분으로 간주하지 않는다.

In [9]:
class FirstClass:
    def __init__(self, value):
        self.value = value

class SecondClass:
    def __init__(self, value):
        self.value = value

second = SecondClass(5)
first = FirstClass(second)

del FirstClass
del SecondClass

```python
# ex14.py

class FirstClass:
    def __init__(self, value: SecondClass) -> None:
        self.value = value

class SecondClass:
    def __init__(self, value: int) -> None:
        self.value = value

second = SecondClass(5)
first = FirstClass(second)
```

```shell
$ python -m mypy --strict item_90_example_14.py
Success: no issues found in 1 source file
```

In [10]:
try:
    class FirstClass:
        def __init__(self, value: SecondClass) -> None:  # Breaks
            self.value = value
    
    class SecondClass:
        def __init__(self, value: int) -> None:
            self.value = value
    
    second = SecondClass(5)
    first = FirstClass(second)
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-10-db8e75e8fc35>", line 2, in <module>
    class FirstClass:
  File "<ipython-input-10-db8e75e8fc35>", line 3, in FirstClass
    def __init__(self, value: SecondClass) -> None:  # Breaks
NameError: name 'SecondClass' is not defined


In [11]:
class FirstClass:
    def __init__(self, value: 'SecondClass') -> None:  # OK
        self.value = value

class SecondClass:
    def __init__(self, value: int) -> None:
        self.value = value

second = SecondClass(5)
first = FirstClass(second)

```python
# ex17.py

from __future__ import annotations

class FirstClass:
    def __init__(self, value: SecondClass) -> None:  # OK
        self.value = value

class SecondClass:
    def __init__(self, value: int) -> None:
        self.value = value

second = SecondClass(5)
first = FirstClass(second)
```

```shell
$ python -m mypy --strict item_90_example_17.py
Success: no issues found in 1 source file

$ python item_90_example_17.py
```

> - 파이썬은 변수, 필드, 함수, 메서드에 타입 정보를 추가할 수 있게 특별한 구문과 `typing` 내장 모듈을 제공한다.
> - 정적인 타입 검사기를 사용하면 타입 정보를 활용해 런타임에 발생할 수 있는 다양한 일반적인 오류를 방지할 수 있다.
> - 타입을 프로그램에 도입하고, API에 타입을 적용하고, 타입 정보를 추가해도 생산성이 떨어지지 않도록 해주는 다양한 모범 사례가 있다.

> - 일반적인 전략은 타입 애너테이션 없이 코드 작성을 시작하고, 테스트를 작성한 다음, 타입 정보가 유용하게 쓰일 곳에 타입 정보를 추가하는 것이다.
> - 타입 힌트는 API와 같이 코드베이스의 경계에서 가장 중요하다.
> - API의 일부분이 아니지만 코드베이스에서 가장 복잡하고 오류가 발생하기 쉬운 부분에 타입 힌트를 적용해도 유용할 수 있다. 하지만 타입 힌트를 코드의 모든 부분에 100% 적용하는 것은 바람직하지 않다.
> - 자동 빌드와 테스트 시스템의 일부분으로 정적 분석을 포함시켜서 코드베이스에 커밋할 때마다 오류가 없는지 검사해야 한다. 추가로 타입 검사에 사용할 설정을 저장소에 유지해서 협업하는 모든 사람이 똑같은 규칙을 사용하게 해야 한다.
> - 타입을 추가하면서 타입 검사기를 실행하는 일이 중요하다.
> - 마지막으로 타입 애너테이션이 필요한 경우가 그리 많지 않을 것이라는 사실을 알아두자.